In [1]:
import numpy as np
import torch


torch.cuda.is_available()

True

In [2]:
# I hate python
import sys
import os


sys.path.insert(0, os.path.abspath("qagnn"))

In [18]:
import json
from munch import DefaultMunch
from modeling.modeling_qagnn import LM_QAGNN, LM_QAGNN_DataLoader


OUTPUT_DIR = "qagnn/saved_models/csqa/enc-FacebookAI/roberta-large__k5__gnndim200__bs64__seed0__20240320_173213"

# Load the configuration from the JSON file
with open(f'{OUTPUT_DIR}/config.json', 'r') as f:
    config = json.load(f)

# Fix the paths (I hate python x2)
paths_to_fix = ["inhouse_train_qids", "train_statements", "dev_statements", "test_statements", "train_adj", "dev_adj", "test_adj", "save_dir"]
for path in paths_to_fix:
    config[path] = f"qagnn/{config[path]}" 

print(config)

relative_ent_emb_path: str = config["ent_emb_paths"][0]
config["ent_emb_paths"][0] = f"qagnn/{relative_ent_emb_path}" 

# Convert dict to object (I hate python x3)
args = DefaultMunch.fromDict(config)


# Now that we've worked around Python's BS, we can finally play around with this
# Now let's load it!
cp_emb = [np.load(path) for path in args.ent_emb_paths]
cp_emb = torch.tensor(np.concatenate(cp_emb, 1), dtype=torch.float)

concept_num, concept_dim = cp_emb.size(0), cp_emb.size(1)
print('| num_concepts: {} |'.format(concept_num))

if torch.cuda.device_count() >= 2 and args.cuda:
    device0 = torch.device("cuda:0")
    device1 = torch.device("cuda:1")
elif torch.cuda.device_count() == 1 and args.cuda:
    device0 = torch.device("cuda:0")
    device1 = torch.device("cuda:0")
else:
    device0 = torch.device("cpu")
    device1 = torch.device("cpu")


dataset = LM_QAGNN_DataLoader(args, args.train_statements, args.train_adj,
                                               args.dev_statements, args.dev_adj,
                                               args.test_statements, args.test_adj,
                                               batch_size=args.batch_size, eval_batch_size=args.eval_batch_size,
                                               device=(device0, device1),
                                               model_name=args.encoder,
                                               max_node_num=args.max_node_num, max_seq_length=args.max_seq_len,
                                               is_inhouse=args.inhouse, inhouse_train_qids_path=args.inhouse_train_qids,
                                               subsample=args.subsample, use_cache=args.use_cache)

###################################################################################################
#   Build model                                                                                   #
###################################################################################################
print('args.num_relation', args.num_relation)
qa_gnn = LM_QAGNN(args, args.encoder, k=args.k, n_ntype=4, n_etype=args.num_relation, n_concept=concept_num,
                                   concept_dim=args.gnn_dim,
                                   concept_in_dim=concept_dim,
                                   n_attention_head=args.att_head_num, fc_dim=args.fc_dim, n_fc_layer=args.fc_layer_num,
                                   p_emb=args.dropouti, p_gnn=args.dropoutg, p_fc=args.dropoutf,
                                   pretrained_concept_emb=cp_emb, freeze_ent_emb=args.freeze_ent_emb,
                                   init_range=args.init_range,
                                   encoder_config={})

qa_gnn.encoder.to(device0)
qa_gnn.decoder.to(device1)

qa_gnn

{'ent_emb': ['tzw'], 'dataset': 'csqa', 'inhouse': True, 'inhouse_train_qids': 'qagnn/data/csqa/inhouse_split_qids.txt', 'train_statements': 'qagnn/data/csqa/statement/train.statement.jsonl', 'dev_statements': 'qagnn/data/csqa/statement/dev.statement.jsonl', 'test_statements': 'qagnn/data/csqa/statement/test.statement.jsonl', 'max_seq_len': 100, 'encoder': 'FacebookAI/roberta-large', 'encoder_layer': -1, 'encoder_lr': 1e-05, 'loss': 'cross_entropy', 'optim': 'radam', 'lr_schedule': 'fixed', 'batch_size': 64, 'warmup_steps': 150, 'max_grad_norm': 1.0, 'weight_decay': 0.01, 'n_epochs': 15, 'max_epochs_before_stop': 10, 'log_interval': 10, 'cuda': True, 'seed': 0, 'debug': False, 'mode': 'train', 'save_dir': 'qagnn/saved_models/csqa/enc-FacebookAI/roberta-large__k5__gnndim200__bs64__seed0__20240320_173213', 'save_model': True, 'load_model_path': None, 'num_relation': 38, 'train_adj': 'qagnn/data/csqa/graph/train.graph.adj.pk', 'dev_adj': 'qagnn/data/csqa/graph/dev.graph.adj.pk', 'test_adj

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1221/1221 [00:00<00:00, 1423.91it/s]


num_choice 5
| ori_adj_len: mu 121.54 sigma 94.04 | adj_len: 107.96 | prune_rate： 0.17 | qc_num: 7.43 | ac_num: 2.07 |
| ori_adj_len: mu 118.44 sigma 90.55 | adj_len: 106.55 | prune_rate： 0.15 | qc_num: 7.20 | ac_num: 2.05 |


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1140/1140 [00:00<00:00, 2022.72it/s]


| ori_adj_len: mu 119.20 sigma 93.49 | adj_len: 106.22 | prune_rate： 0.16 | qc_num: 7.38 | ac_num: 2.05 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LM_QAGNN(
  (encoder): TextEncoder(
    (module): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_featu

In [20]:
isinstance(qa_gnn, torch.nn.Module)

True

In [21]:
# Load the checkpoint
checkpoint = torch.load(f"{OUTPUT_DIR}/model.pt.14")

In [23]:
checkpoint

[OrderedDict([('encoder.module.embeddings.word_embeddings.weight',
               tensor([[-0.1402, -0.0094,  0.0387,  ...,  0.0506, -0.0055, -0.0362],
                       [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156],
                       [-0.0830, -0.0004, -0.1170,  ...,  0.1088,  0.0691, -0.0357],
                       ...,
                       [ 0.0393,  0.0031,  0.0465,  ..., -0.0240, -0.0505,  0.0342],
                       [ 0.0499,  0.0272,  0.0413,  ..., -0.0370, -0.0100,  0.0071],
                       [-0.0149, -0.0114, -0.0222,  ...,  0.0441,  0.0116, -0.0330]],
                      device='cuda:0')),
              ('encoder.module.embeddings.position_embeddings.weight',
               tensor([[-0.0038,  0.0253, -0.0092,  ...,  0.0177,  0.0062, -0.0162],
                       [ 0.0117, -0.0019, -0.0267,  ...,  0.0062, -0.0193,  0.0263],
                       [ 0.0319,  0.0150, -0.0553,  ..., -0.0719, -0.0456,  0.0466],
                       ...,

In [24]:
len(checkpoint)

2

In [27]:
checkpoint[0]

OrderedDict([('encoder.module.embeddings.word_embeddings.weight',
              tensor([[-0.1402, -0.0094,  0.0387,  ...,  0.0506, -0.0055, -0.0362],
                      [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156],
                      [-0.0830, -0.0004, -0.1170,  ...,  0.1088,  0.0691, -0.0357],
                      ...,
                      [ 0.0393,  0.0031,  0.0465,  ..., -0.0240, -0.0505,  0.0342],
                      [ 0.0499,  0.0272,  0.0413,  ..., -0.0370, -0.0100,  0.0071],
                      [-0.0149, -0.0114, -0.0222,  ...,  0.0441,  0.0116, -0.0330]],
                     device='cuda:0')),
             ('encoder.module.embeddings.position_embeddings.weight',
              tensor([[-0.0038,  0.0253, -0.0092,  ...,  0.0177,  0.0062, -0.0162],
                      [ 0.0117, -0.0019, -0.0267,  ...,  0.0062, -0.0193,  0.0263],
                      [ 0.0319,  0.0150, -0.0553,  ..., -0.0719, -0.0456,  0.0466],
                      ...,
             

In [25]:
checkpoint[1]

Namespace(att_head_num=2, batch_size=64, cuda=True, dataset='csqa', debug=False, decoder_lr=0.001, dev_adj='data/csqa/graph/dev.graph.adj.pk', dev_statements='data/csqa/statement/dev.statement.jsonl', drop_partial_batch=False, dropoutf=0.2, dropoutg=0.2, dropouti=0.2, encoder='FacebookAI/roberta-large', encoder_layer=-1, encoder_lr=1e-05, ent_emb=['tzw'], ent_emb_paths=['data/cpnet/tzw.ent.npy'], eval_batch_size=2, fc_dim=200, fc_layer_num=0, fill_partial_batch=False, fp16=True, freeze_ent_emb=True, gnn_dim=200, inhouse=True, inhouse_train_qids='data/csqa/inhouse_split_qids.txt', init_range=0.02, k=5, load_model_path=None, log_interval=10, loss='cross_entropy', lr_schedule='fixed', max_epochs_before_stop=10, max_grad_norm=1.0, max_node_num=200, max_seq_len=100, mini_batch_size=2, mode='train', n_epochs=15, num_relation=38, optim='radam', refreeze_epoch=10000, save_dir='saved_models/csqa/enc-FacebookAI/roberta-large__k5__gnndim200__bs64__seed0__20240320_173213', save_model=True, seed=0,

In [28]:
# load this mf
qa_gnn.load_state_dict(checkpoint[0])

<All keys matched successfully>

In [ ]:
if args.load_model_path:
            print (f'loading and initializing model from {args.load_model_path}')
            model_state_dict, old_args = torch.load(args.load_model_path, map_location=torch.device('cpu'))
            model.load_state_dict(model_state_dict)

        model.encoder.to(device0)
        model.decoder.to(device1)

In [30]:
dir(dataset)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'args',
 'batch_size',
 'dev',
 'dev_adj_data',
 'dev_decoder_data',
 'dev_encoder_data',
 'dev_labels',
 'dev_qids',
 'dev_size',
 'device0',
 'device1',
 'eval_batch_size',
 'inhouse_test_indexes',
 'inhouse_train_indexes',
 'is_inhouse',
 'num_choice',
 'test',
 'test_adj_data',
 'test_decoder_data',
 'test_encoder_data',
 'test_labels',
 'test_qids',
 'test_size',
 'train',
 'train_adj_data',
 'train_decoder_data',
 'train_encoder_data',
 'train_eval',
 'train_labels',
 'train_qids',
 'train_size']

In [33]:
# Tokenization
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")
tokenizer.encode("yo yo yo dfja;ldfja;dj akdjfjfhdf hiiii hi", add_special_tokens=False)

[9839,
 25610,
 25610,
 47942,
 1910,
 131,
 4779,
 506,
 1910,
 131,
 26029,
 18735,
 26029,
 506,
 267,
 506,
 298,
 36807,
 1368,
 4132,
 4132,
 20280]

In [146]:
from datasets import load_dataset


csqa = load_dataset("tau/commonsense_qa")
csqa

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1140
    })
})

In [155]:
csqa["train"]["question"].index('Bill is stuck in marsh when a man comes up to him peaking Cajun, where is he?')

15

In [206]:
csqa["train"]["question"]

['The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
 'Sammy wanted to go to where the people were.  Where might he go?',
 'To locate a choker not located in a jewelry box or boutique where would you go?',
 'Google Maps and other highway and street GPS services have replaced what?',
 'The fox walked from the city into the forest, what was it looking for?',
 'What home entertainment equipment requires cable?',
 'The only baggage the woman checked was a drawstring bag, where was she heading with it?',
 'The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?',
 'What do people use to absorb extra ink from a fountain pen?',
 'Where is a business restaurant likely to be located?',
 'Where do you put your grapes just before checking out?',
 'Before getting a divorce, what did the wife feel who was doing all the work?',
 'Johnny sat on a bench and relaxed after doing a lot of 

In [157]:
csqa["train"][15]["id"]

'e6766a66f8d326bf00fbf628a0e4ef24'

In [215]:
import re


def remove_non_alphanumeric(text):
  """
  Removes all non-alphanumeric characters from a string.

  Args:
    text: The string to process.

  Returns:
    The string with all non-alphanumeric characters removed.
  """

  # Use a regular expression to match all non-alphanumeric characters
  pattern = r'\W+'

  # Replace all matches with an empty string
  return re.sub(pattern, '', text)


# CommonSenseQA Question ID

# dataset is either train, dev, or test from csqa huggingface
# return None if not found
def question_to_qid_from_data(question: str, dataset) -> str:
    try:
        q_index: int = dataset["question"].index(question)
        qid = dataset[q_index]["id"]

        return qid
    except:  # question not found
        # Try a more accurate method of search
        # remove whitespace and special characters
        question_bare: str = remove_non_alphanumeric(question)
        #print(f"question_bare: {question_bare}")
        questions: List[str] = dataset["question"]
        for (i, q) in enumerate(questions):
            q_bare: str = remove_non_alphanumeric(q)
            #print(f"  q_bare: {q_bare}")
            if q_bare == question_bare:
                qid = dataset[i]["id"]
                return qid

        # Ok the question for sure is not found
        return None


def question_to_qid(question: str):
    qid = question_to_qid_from_data(question, csqa["train"])

    if qid is not None:
        return qid

    qid = question_to_qid_from_data(question, csqa["validation"])

    if qid is not None:
        return qid

    qid = question_to_qid_from_data(question, csqa["test"])

    if qid is not None:
        return qid

    return None


# dataset is either train, dev, or test
# return None if not found
def qid_to_input_from_dataset(qid, dataset):
    def index_preserve_dims(data_elem, index: int):
        if type(data_elem) is list:
            return [data_elem[index]]
        else:  # torch tensor
            return data_elem[index].unsqueeze(0)
        
    
    for qids, labels, *input_data in dataset:
        #batch_size: int = len(qids)
        try:
            index: int = qids.index(qid)

            label_index: int = labels[index]
            input_data_from_qid = [index_preserve_dims(input_data_element, index) for input_data_element in input_data]

            return (label_index, input_data_from_qid)
        except:  # not in the list
            continue
    
    return None


def qid_to_input(qid):
    input_ = qid_to_input_from_dataset(qid, dataset.train())

    if input_ is not None:
        return input_

    input_ = qid_to_input_from_dataset(qid, dataset.dev())

    if input_ is not None:
        return input_

    input_ = qid_to_input_from_dataset(qid, dataset.test())

    if input_ is not None:
        return input_

    return None


def question_to_input(question: str):
    qid = question_to_qid(question)
    return qid_to_input(qid)  # returns a tuple (label_idx, input_data_from_qid)

In [216]:
#print(question_to_input("Learning languages is difficult.  Even in a full immersion environment you're likely to make mistakes, mistakes will cause misinterpretation, which will be uncomfortable, which will cause what?,"))
print(question_to_input("Learning languages is difficult. Even in a full immersion environment you're likely to make mistakes, mistakes will cause misinterpretation, which will be uncomfortable, which will cause what?"))

(tensor(2, device='cuda:0'), [tensor([[[    0, 42489, 11991,    16,  1202,     4,  1437,  1648,    11,    10,
            455, 35239,  1737,    47,   214,   533,     7,   146,  6160,     6,
           6160,    40,  1303, 36939,  1258,     6,    61,    40,    28,  9800,
              6,    61,    40,  1303,    99, 33647,     2,     2,   357,  4358,
              2,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [    0, 42489, 11991,    16,  1202,     4,  1437,  1648,    11,    10,
            455, 35239,  1737,    47,   214,   533,     7,   146,  6160,     6,
         

In [217]:
# Validate the previous

from typing import List


(label_idx, local_input_data) = question_to_input("Where does a person need to get when they have more kids?")

for i in range(len(local_input_data)):
        try:
            tensor: torch.Tensor = local_input_data[i]
            print(f"{tensor.shape}, dtype={tensor.dtype}, device={tensor.device}")
            #print(str(input_data[i].int())[:128])
        except:
            tensors: List = local_input_data[i]
            #print(input_data[i])
            #list_tensor = torch.tensor(input_data[i], device=device0)
            #print(f"{list_tensor.shape}, dtype={input_data[i].dtype}")
            
            print(f"{[len(tensors)]}")

torch.Size([1, 5, 100]), dtype=torch.int64, device=cuda:0
torch.Size([1, 5, 100]), dtype=torch.int64, device=cuda:0
torch.Size([1, 5, 100]), dtype=torch.int64, device=cuda:0
torch.Size([1, 5, 100]), dtype=torch.bool, device=cuda:0
torch.Size([1, 5, 200]), dtype=torch.int64, device=cuda:0
torch.Size([1, 5, 200]), dtype=torch.int64, device=cuda:0
torch.Size([1, 5, 200, 1]), dtype=torch.float32, device=cuda:0
torch.Size([1, 5]), dtype=torch.int64, device=cuda:0
[1]
[1]


In [184]:
qa_gnn.encoder.to(device0)
qa_gnn.decoder.to(device1)

QAGNN(
  (concept_emb): CustomizedEmbedding(
    (emb): Embedding(799273, 1024)
    (cpt_transform): Linear(in_features=1024, out_features=200, bias=True)
    (activation): GELU()
  )
  (svec2nvec): Linear(in_features=1024, out_features=200, bias=True)
  (activation): GELU()
  (gnn): QAGNN_Message_Passing(
    (emb_node_type): Linear(in_features=4, out_features=100, bias=True)
    (emb_score): Linear(in_features=100, out_features=100, bias=True)
    (edge_encoder): Sequential(
      (0): Linear(in_features=47, out_features=200, bias=True)
      (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=200, out_features=200, bias=True)
    )
    (gnn_layers): ModuleList(
      (0-4): 5 x GATConvE()
    )
    (Vh): Linear(in_features=200, out_features=200, bias=True)
    (Vx): Linear(in_features=200, out_features=200, bias=True)
    (activation): GELU()
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pooler):

In [218]:
POSSIBLE_ANSWERS = ['A', 'B', 'C', 'D', 'E']

# Make predictions
# question: just the question, not the prompt given to LLMs
def qa_gnn_predict(question: str, return_letter: bool = False):
    result = {}
    
    (label_idx, input_data) = question_to_input(question)
    
    actual_answer: str = POSSIBLE_ANSWERS[label_idx]
    result["actual_answer"] = actual_answer
    
    # run inference
    with torch.no_grad():
        logits, _, concept_ids, node_type_ids, edge_index, edge_type = qa_gnn(*input_data, detail=True)
        prediction = logits.argmax(1)  # [batch_size which should = 1,]
        #print(predictions.shape)
        #print(POSSIBLE_ANSWERS[predictions])

        
        if return_letter:
            answer_choice = POSSIBLE_ANSWERS[prediction]
            result["prediction"] = answer_choice
        else:
            result["prediction"] = prediction

    return result


qa_gnn_predict("Where does a person need to get when they have more kids?", return_letter=True)

{'actual_answer': 'E', 'prediction': 'E'}

In [219]:
sample_prompt = '''
The hippy wanted peace with all humans, he had strong what for them all?
  A.  "names"
  B. "words"
  C. "naval"
  D. "bladders"
  E. "feelings"
'''[1:-1]

qa_gnn_predict("The hippy wanted peace with all humans, he had strong what for them all?", return_letter=True)

{'actual_answer': 'E', 'prediction': 'E'}

# Hierarchical Knowledge Graph Reasoning Evaluation

In [223]:
# 50 samples of hierarchical reasoning from CommonSenseQA
import pandas as pd


df = pd.read_csv("qagnn/data/CommonSenseQA_hierarchical_samples.csv")
df.head()

,Question,Answers,Answer
0,"Before getting a divorce, what did the wife fe...","{ ""label"": [ ""A"", ""B"", ""C"", ""D"", ""E"" ], ""text""...",C
1,The sensor would just the distance then set of...,"{ ""label"": [ ""A"", ""B"", ""C"", ""D"", ""E"" ], ""text""...",D
2,Learning languages is difficult. Even in a ful...,"{ ""label"": [ ""A"", ""B"", ""C"", ""D"", ""E"" ], ""text""...",C
3,The sanctions against the school were a punish...,"{ ""label"": [ ""A"", ""B"", ""C"", ""D"", ""E"" ], ""text""...",A
4,To locate a choker not located in a jewelry bo...,"{ ""label"": [ ""A"", ""B"", ""C"", ""D"", ""E"" ], ""text""...",A


In [224]:
from typing import List, Dict


POSSIBLE_ANSWERS = ["A", "B", "C", "D", "E"]


EXTRACTION_START_INDEX = len('{ "label": [ "A", "B", "C", "D", "E" ], "text": ')
def extract_answers_list(answers_entry: str) -> List[str]:  # Get the representation of each of these answers
  return answers_entry[EXTRACTION_START_INDEX:-2].strip('][').split(', ')


def get_mcq(q_idx: int) -> Dict:
  mcq = df.iloc[q_idx]

  question: str = mcq["Question"]
  answers: List[str] = extract_answers_list(mcq["Answers"])
  real_answer: int = POSSIBLE_ANSWERS.index(mcq["Answer"])

  return {"Q": question, "Possible Answers": answers, "AnswerIndex": real_answer}


def get_random_mcq() -> Dict:
  return get_mcq(np.random.randint(0, len(df)))


def mcq_to_prompt(mcq: Dict, include_answer: bool = True) -> str:
  answer_options: List[str] = mcq["Possible Answers"]

  answer: str = ""
  if include_answer:
    answer = POSSIBLE_ANSWERS[mcq["AnswerIndex"]]

  return f"""
  Question: {mcq["Q"]}
  A. {answer_options[0]}
  B. {answer_options[1]}
  C. {answer_options[2]}
  D. {answer_options[3]}
  E. {answer_options[4]}
  Answer: {answer}
  """[1:-1]


get_random_mcq()

{'Q': 'James thought that giving the AI a secular upbringing would be the better choice. He felt that the alternative might have results that were too what?',
 'Possible Answers': [' "eternal"',
  '"religious"',
  '"unpredictable"',
  '"holy"',
  '"monastic" '],
 'AnswerIndex': 2}

In [237]:
def eval_qa_gnn() -> pd.DataFrame:
    eval_df: pd.DataFrame = pd.DataFrame(columns=["Question", "Possible Answers", "Real Answer", "Predicted Answer"])

    for i in range(len(df)):
        mcq: Dict = get_mcq(i)
        question: str = mcq["Q"]

        print(question, end=" ")
        print(mcq["Possible Answers"])
        
        eval_df.loc[i, "Question"] = question 
        eval_df.loc[i, "Possible Answers"] = str(mcq["Possible Answers"])

        result = qa_gnn_predict(question, return_letter=True)
        prediction: str = result["prediction"]
        actual_answer: str = result["actual_answer"]
        
        pred_is_correct: bool = (prediction == actual_answer)
        print(f"Answer: {prediction}; Real Answer: {actual_answer}", end="\n\n")
        
        eval_df.loc[i, "Real Answer"] = actual_answer
        eval_df.loc[i, "Predicted Answer"] = prediction

        eval_df.loc[i, "Correct"] = pred_is_correct

    return eval_df


def summary_eval_df(eval_df: pd.DataFrame, name: str, idx=0) -> pd.DataFrame:
    summary_df: pd.DataFrame = pd.DataFrame(columns=["Correct", "Incorrect", "Accuracy"])

    summary_df.loc[idx, "Model"] = name
    summary_df.loc[idx, "Correct"] = eval_df["Correct"].value_counts()[True]
    summary_df.loc[idx, "Incorrect"] = eval_df["Correct"].value_counts()[False]
    summary_df.loc[idx, "Accuracy"] = eval_df["Correct"].sum() / len(eval_df)

    return summary_df


def full_eval_qa_gnn() -> Dict:
    eval_df: pd.DataFrame = eval_qa_gnn()
    summary_df: pd.DataFrame = summary_eval_df(eval_df, name="QA-GNN")
    
    return {"evaluation": eval_df, "summary": summary_df}

In [238]:
print("QA-GNN Evaluation on Hierarchical Knowledge Graph Reasoning")
eval_qagnn = full_eval_qa_gnn()

eval_qagnn["summary"]

QA-GNN Evaluation on Hierarchical Knowledge Graph Reasoning
Before getting a divorce, what did the wife feel who was doing all the work? [' "harder"', '"anguish"', '"bitterness"', '"tears"', '"sadness" ']
Answer: C; Real Answer: C

The sensor would just the distance then set off an alarm, the installation expert explained it was called a what kind of sensor? [' "near"', '"closeness"', '"here"', '"proximity"', '"this" ']
Answer: D; Real Answer: D

Learning languages is difficult. Even in a full immersion environment you're likely to make mistakes, mistakes will cause misinterpretation, which will be uncomfortable, which will cause what? [' "better communication"', '"overthinking"', '"frustration"', '"misunderstandings"', '"headaches" ']
Answer: C; Real Answer: C

The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? [' "ignore"', '"enforce"', '"authoritarian"', '"yell at"', '"avoid" ']
Answer: A; Real Answer: A

To loc

,Correct,Incorrect,Accuracy,Model
0,45,4,0.918367,QA-GNN


In [244]:
eval_qagnn["summary"] = eval_qagnn["summary"].reindex(columns=["Model", "Correct", "Incorrect", "Accuracy"])  # reorder
eval_qagnn["summary"]

,Model,Correct,Incorrect,Accuracy
0,QA-GNN,45,4,0.918367


In [245]:
eval_qagnn["evaluation"]

,Question,Possible Answers,Real Answer,Predicted Answer,Correct
0,"Before getting a divorce, what did the wife fe...","[' ""harder""', '""anguish""', '""bitterness""', '""t...",C,C,True
1,The sensor would just the distance then set of...,"[' ""near""', '""closeness""', '""here""', '""proximi...",D,D,True
2,Learning languages is difficult. Even in a ful...,"[' ""better communication""', '""overthinking""', ...",C,C,True
3,The sanctions against the school were a punish...,"[' ""ignore""', '""enforce""', '""authoritarian""', ...",A,A,True
4,To locate a choker not located in a jewelry bo...,"[' ""jewelry store""', '""neck""', '""jewlery box""'...",A,A,True
5,"The fox walked from the city into the forest, ...","[' ""pretty flowers.""', '""hen house""', '""natura...",C,C,True
6,Joe suffered many consequences from stabbing a...,"[' ""knife wounds""', '""buy a gun""', '""bleeding""...",E,E,True
7,What could happen to a paper if you leave it o...,"[' ""park""', '""make time for""', '""receive instr...",C,C,True
8,"The drug kingpin told his man to run errands, ...","[' ""park""', '""make time for""', '""receive instr...",D,D,True
9,Too many people want exotic snakes. The demand...,"[' ""ditch""', '""shop""', '""north america""', '""pe...",D,D,True


In [246]:
# Load the LLM stats
# from https://drive.google.com/file/d/1HIFetTdjmL539WO3KCnk2jNNEgYU2dOb/view?usp=sharing
llm_summaries_df = pd.read_csv("qagnn/data/llm_data/llm_summaries_df.csv")

falcon_eval_df = pd.read_csv("qagnn/data/llm_data/Falcon-7b-instruct_eval_df.csv")
gemma_eval_df = pd.read_csv("qagnn/data/llm_data/Gemma-7b-it_eval_df.csv")
llama2_eval_df = pd.read_csv("qagnn/data/llm_data/Llama-2-7b-chat-hf_eval_df.csv")
mistral_eval_df = pd.read_csv("qagnn/data/llm_data/Mistral-7B-Instruct-v0.2_eval_df.csv")

llm_summaries_df

,Model,Correct,Incorrect,Accuracy
0,Gemma-7b-it,21,28,0.428571
1,Llama-2-7b-chat-hf,17,32,0.346939
2,Mistral-7B-Instruct-v0.2,37,12,0.755102
3,Falcon-7b-instruct,13,36,0.265306


# Final Results

In [247]:
eval_qagnn_summary: pd.DataFrame = eval_qagnn["summary"]
eval_qagnn_evaluation: pd.DataFrame = eval_qagnn["evaluation"]

summaries_df: pd.DataFrame = pd.concat([eval_qagnn_summary, llm_summaries_df])
summaries_df

,Model,Correct,Incorrect,Accuracy
0,QA-GNN,45,4,0.918367
0,Gemma-7b-it,21,28,0.428571
1,Llama-2-7b-chat-hf,17,32,0.346939
2,Mistral-7B-Instruct-v0.2,37,12,0.755102
3,Falcon-7b-instruct,13,36,0.265306


In [248]:
# Export this
summaries_df.to_csv("qagnn/data/summaries_df.csv")

# Stray Experimentation

In [172]:
# This cell was used for tons of experimentation for reverse engineering

from typing import List


POSSIBLE_ANSWERS = ['A', 'B', 'C', 'D', 'E']

i = 0
stop = 1

# data: qids, labels, *input_data (len == 10)
# iterate batch-by-batch
for qids, labels, *input_data in dataset.train():
    # FINDINGS
    batch_size: int = len(qids)
    # qids == question_ids???
    mcq_labels: List[int] = list(labels)  # they will go up to 4
    mcq_answers: List[str] = [POSSIBLE_ANSWERS[mcq_label] for mcq_label in mcq_labels]
    num_choice: int = 5
    #d_sent: int = 100
    #n_node: int = 200
    
    #print(qids)
    #print(mcq_answers)
    #print(type(input_data))
    #print(type(input_data[-1]))
    #print(input_data[7][0].unsqueeze(0).shape)
    #print(f"batch_size: {batch_size}")
    # input_data: list; len == 10
    
    # [0:4]: sent_vecs (batch_size, num_choice, d_sent)  -> (batch_size * num_choice, d_sent)
    
    # [4]: concept_ids (batch_size, num_choice, n_node)  -> (batch_size * num_choice, n_node)
    # [5]: node_type_ids (batch_size, num_choice, n_node) -> (batch_size * num_choice, n_node)
    # [6]: ???
    # [7]: adj_lengths (batch_size, num_choice)          -> (batch_size * num_choice, )
    
    # [8]: edge_index_orig
    # [9]: edge_type_orig
    
    # tests
    #print(input_data[0].dtype)
    #entity = torch.tensor([input_data[0].int(), input_data[1].int(), input_data[2].int(), input_data[3].int()], device=device0)
    
    for i in range(len(input_data)):
        try:
            print(f"{input_data[i].shape}, dtype={input_data[i].dtype}")
            #print(str(input_data[i].int())[:128])
        except:
            #print(input_data[i])
            #list_tensor = torch.tensor(input_data[i], device=device0)
            #print(f"{list_tensor.shape}, dtype={input_data[i].dtype}")
            tensors: List = input_data[i]
            print(f"{[len(tensors)]}")
            for tensor in tensors:
                try:
                    print(f"   {tensor.shape}, dtype={tensor.dtype}")
                    #print(str(input_data[i].int())[:128])
                except:
                    #print(input_data[i])
                    #list_tensor = torch.tensor(input_data[i], device=device0)
                    #print(f"{list_tensor.shape}, dtype={input_data[i].dtype}")
                    print(f"   {type(tensor)}", end=" ")
                    print(f"{[len(tensor)]}")
            
    #print(entity.shape)  # [64, 5, 100]
    #print(entity)
    #print(qids)
    #print(labels)
    
    #print(len(qids))
    #print(input_data[6])
    
    #print(len(entity))
    #print(type(entity))

    #print(tokenizer.decode(entity, skip_special_tokens=True))
    
    i += 1
    if i >= stop:
        break

torch.Size([64, 5, 100]), dtype=torch.int64
torch.Size([64, 5, 100]), dtype=torch.int64
torch.Size([64, 5, 100]), dtype=torch.int64
torch.Size([64, 5, 100]), dtype=torch.bool
torch.Size([64, 5, 200]), dtype=torch.int64
torch.Size([64, 5, 200]), dtype=torch.int64
torch.Size([64, 5, 200, 1]), dtype=torch.float32
torch.Size([64, 5]), dtype=torch.int64
[64]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <class 'list'> [5]
   <cl